# Question
Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, you will need to use one-versus-the-rest to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?

# Problem Definition

To solve this problem, we will follow these steps:

1. Load the MNIST dataset.
2. Split the dataset into training and validation sets.
3. Use one-versus-the-rest (OvR) to train SVM classifiers for each digit.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [3]:
from sklearn.svm import LinearSVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

LinearSVC(random_state=42)

In [4]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.9275666666666667

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [6]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

g:\Programming exercise\python exercise\machine learning ex\ml_env\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [7]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9275166666666667

In [8]:
from sklearn.svm import SVC
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

In [9]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9455333333333333

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=5.478753659498475, gamma=0.04168321964818925; total time=   0.4s
[CV] END .....C=5.478753659498475, gamma=0.04168321964818925; total time=   0.4s
[CV] END .....C=5.478753659498475, gamma=0.04168321964818925; total time=   0.4s
[CV] END ...C=6.597717032890341, gamma=0.0032952646243389304; total time=   0.3s
[CV] END ...C=6.597717032890341, gamma=0.0032952646243389304; total time=   0.3s
[CV] END ...C=6.597717032890341, gamma=0.0032952646243389304; total time=   0.3s
[CV] END .....C=10.067142807331368, gamma=0.0685684125633251; total time=   0.4s
[CV] END .....C=10.067142807331368, gamma=0.0685684125633251; total time=   0.4s
[CV] END .....C=10.067142807331368, gamma=0.0685684125633251; total time=   0.4s
[CV] END ..C=4.2480868013968145, gamma=0.0026338099981722284; total time=   0.5s
[CV] END ..C=4.2480868013968145, gamma=0.0026338099981722284; total time=   0.4s
[CV] END ..C=4.2480868013968145, gamma=0.0026338

RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000193BB1041A0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000193BA80FA70>},
                   verbose=2)

In [11]:
rnd_search_cv.best_estimator_

SVC(C=np.float64(4.2480868013968145), gamma=np.float64(0.0026338099981722284))

In [12]:
rnd_search_cv.best_score_

np.float64(0.8329796862730995)

In [13]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=np.float64(4.2480868013968145), gamma=np.float64(0.0026338099981722284))

In [14]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.99945

In [15]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9676